![Logo  deteccao de doencas do algodao](CottonDiseasePrediction.png)

# Detecção de Doenças em Algodão

Bem-vindo ao meu projeto de detecção de doenças em plantações de algodão! Neste notebook, exploraremos uma abordagem baseada em visão computacional e aprendizado de máquina para identificar e classificar se a imagem pertence ou não à classe doente.

### Objetivo do Projeto

O principal objetivo deste projeto é desenvolver um sistema de detecção simples e preciso que possa ajudar agricultores a identificar rapidamente problemas de saúde em suas plantações de algodão. A detecção precoce de doenças é crucial para a implementação de medidas preventivas e o controle efetivo, contribuindo para uma produção saudável e sustentável.

### Metodologia

Utilizaremos técnicas avançadas de processamento de imagens e machine learning, mais especificamente CNN, para treinar um modelo capaz de reconhecer padrões associados a diferentes doenças em folhas de algodão. Este projeto se concentrará em técnicas de classificação para implementação e avaliação do modelo.

### Conjunto de Dados

Para alcançar nossos objetivos, utilizaremos um conjunto de dados abrangente, contendo imagens rotuladas de folhas de algodão saudáveis e afetadas por diversas doenças.

### Estrutura do Notebook

1. Bibliotecas
2. Treinamento e DataLoader
3. Construindo a CNN
4. Resumo
5. Treinamento da CNN
6. Teste
7. Conclusão

Vamos começar!



## Configurando o Ambiente

In [2]:
!nvidia-smi

Sun Dec 10 15:02:45 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.12                 Driver Version: 546.12       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   56C    P8               7W /  45W |     69MiB /  6144MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("GPU não disponível. Usando CPU.")

ModuleNotFoundError: No module named 'torch'

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8551858905635952306
xla_global_id: -1
]


### 1. Bibliotecas

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D
import numpy as np
from tensorflow.keras.preprocessing import image

print(tf.__version__)

### 2. Treinamento e DataLoader

In [34]:
# Configurando gerador de imagens para aumentação de dados.
train_datagen = ImageDataGenerator(rescale = 1./255,           # Normalização dos valores dos pixels para intervalo entre 0 e 1.
                                   shear_range = 0.2,        # Cisalhamento aleatório para criar uma distorção semelhante a uma tesoura.
                                   zoom_range = 0.2,          # Zoom aleatório para adicionar variação à escala.
                                   horizontal_flip = True)   # Inversão horizontal aleatória para adicionar variação refletindo a imagem.

In [35]:
# Configurando o conjunto de treinamento usando o gerador de imagens para aumentação de dados.
training_set = train_datagen.flow_from_directory('Cotton Disease/train',
                                                 target_size=(64, 64),   # Tamanho desejado para as imagens (64x64 pixels).
                                                 batch_size=32,          # Tamanho do lote de imagens utilizado durante o treinamento.
                                                 class_mode='categorical')  # Modo de classificação categórica para problemas de classificação multiclasse.

# Configurando o conjunto de teste usando um gerador de imagens sem aumentação de dados.
test_datagen = ImageDataGenerator(rescale=1./255)  # Apenas normalização para o conjunto de teste.
test_set = test_datagen.flow_from_directory('Cotton Disease/test',
                                            target_size=(64, 64),   # Tamanho desejado para as imagens (64x64 pixels).
                                            batch_size=32,          # Tamanho do lote de imagens utilizado durante a avaliação.
                                            class_mode='categorical')  # Modo de classificação categórica para problemas de classificação multiclasse.


Found 1951 images belonging to 4 classes.
Found 106 images belonging to 4 classes.


### Construindo a Convolutional Neural Network (CNN)

Imagine que estamos treinando um "cérebro artificial", ou rede neural, para reconhecer diferentes tipos de imagens, como gatos, cachorros, carros, ou doenças no algodão, como é o nosso caso.

#### Primeira Etapa - Reconhecimento de Formas:

No início, o cérebro aprende a reconhecer formas básicas nas imagens, como bordas, curvas, ou padrões simples. Isso é feito através de uma fase chamada "convolução", onde o cérebro olha para pequenas partes da imagem de cada vez.

Em seguida, ele dá uma olhada mais ampla para entender as características mais importantes, como a forma geral, usando uma fase chamada "agrupamento".

In [1]:
# Iniciando a CNN
cnn = tf.keras.models.Sequential()

# Passo 1 - Convolução
cnn.add(tf.keras.layers.Conv2D(filters=32,
                               padding="same",
                               kernel_size=3,
                               activation='relu',
                               input_shape=[64, 64, 3]))

NameError: name 'tf' is not defined

#### Segunda Etapa - Compreensão Avançada:

Depois de aprender essas características básicas, o cérebro aprimora sua compreensão, olhando para partes maiores e mais complexas das imagens. Isso é feito adicionando outra camada de "convolução" e "agrupamento".

In [37]:
# Paso 2 - Agrupamento
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adicionando uma seguinda camada convolucvional
cnn.add(tf.keras.layers.Conv2D(filters=32, padding='same', kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))

#### Próximo Passo - Pensamento Abstrato:

Agora, o cérebro precisa pensar de maneira mais abstrata. Ele "esmaga" as informações aprendidas até agora para torná-las mais fáceis de entender, e isso é chamado de "achatamento".

Em seguida, ele conecta todas essas informações da maneira que o cérebro humano faz, para entender padrões mais complexos.

In [38]:
# Passo 3 - Achatamento
cnn.add(tf.keras.layers.Flatten())

# Passo 4 - Conexão completa
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

#### Decisões Finais - Qual é a Imagem?:

Finalmente, o cérebro faz as decisões finais. Com base em tudo o que aprendeu, decide se a imagem é de uma planta doente ou não.

Ele atribui probabilidades a cada opção, o que significa que ele pode não ter certeza absoluta, mas faz a melhor suposição possível.

Em resumo, a Convolutional Neural Network (CNN) é como um cérebro artificial que aprende a reconhecer diferentes coisas em imagens, passo a passo, desde formas simples até padrões mais complexos, para tomar decisões sobre o que está vendo.

In [39]:
# Passo 5 - Camada de saída
cnn.add(tf.keras.layers.Dense(units=4, activation='softmax'))

## Resumo

In [41]:
cnn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 32, 32, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 16, 16, 32)        0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 8192)              0         
                                                                 
 dense_5 (Dense)             (None, 128)              

## Treinando a CNN